In [ ]:
# default_exp tabular

In [ ]:
from fastai.basics import *
from fastai.tabular.all import *
from fastai.callback.all import *
from fastai.distributed import *
from fastprogress import fastprogress
from fastai.callback.mixup import *
from fastscript import *

torch.backends.cudnn.benchmark = True
fastprogress.MAX_COLS = 80

# fastai's Tabular benchmark
> Embdedings and FC layers

In [ ]:
#export
def get_dls(bs=64):
    path = untar_data(URLs.ADULT_SAMPLE)
    dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
        cat_names = ['workclass', 'education', 'marital-status', 'occupation',
                 'relationship', 'race'],
        cont_names = ['age', 'fnlwgt', 'education-num'],
        procs = [Categorify, FillMissing, Normalize],
        bs=bs)
    return dls

In [ ]:
dls = get_dls()

In [ ]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,State-gov,11th,Married-civ-spouse,Adm-clerical,Husband,White,False,49.000000,203039.000250,7.0,<50k
1,Private,Masters,Never-married,Exec-managerial,Not-in-family,White,False,42.000000,210524.999580,14.0,<50k
2,Private,Assoc-acdm,Married-civ-spouse,Prof-specialty,Wife,White,False,48.000000,313924.993803,12.0,>=50k
3,Private,Bachelors,Married-civ-spouse,Protective-serv,Husband,Black,False,33.000000,247205.000913,13.0,<50k
4,Private,1st-4th,Widowed,Handlers-cleaners,Not-in-family,White,False,60.999999,668362.024904,2.0,<50k
5,Private,HS-grad,Married-spouse-absent,Other-service,Unmarried,Black,False,45.000000,234698.998742,9.0,>=50k
6,Private,Prof-school,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,False,59.000000,122283.001076,15.0,>=50k
7,Self-emp-not-inc,HS-grad,Married-civ-spouse,Other-service,Husband,White,False,38.000000,184455.999875,9.0,<50k
8,Local-gov,Masters,Never-married,Prof-specialty,Own-child,White,False,25.000000,276249.001345,14.0,<50k
9,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,False,60.000001,85412.997963,9.0,>=50k


In [ ]:
#export
def train(
    gpu=None,
    bs=64,
    epochs=5,
    fp16=0,
    dump=0,
    runs=1,
):
    "Training of Tabular data 'ADULT_SAMPLE'."

    # gpu = setup_distrib(gpu)
    if gpu is not None: torch.cuda.set_device(gpu)

    path = untar_data(URLs.ADULT_SAMPLE)
    dls = get_dls(bs)

    if not gpu: print(f'epochs: {epochs};')

    for run in range(runs):
        print(f'Run: {run}')

        learn = tabular_learner(dls, metrics=accuracy)
        if dump: print(learn.model); exit()
        if fp16: learn = learn.to_fp16()

        n_gpu = torch.cuda.device_count()
        ctx = learn.parallel_ctx if gpu is None and n_gpu else learn.distrib_ctx

        with partial(ctx, gpu)(): # distributed traing requires "-m fastai.launch"
            print(f"Training in {ctx.__name__} context on GPU {gpu if gpu is not None else list(range(n_gpu))}")
            learn.fit_one_cycle(epochs)

In [ ]:
#export
def train_tabular(gpu=None, bs=64, fp16=0, epochs=5, runs=1):
    train(gpu=gpu, bs=bs, fp16=fp16, epochs=epochs, runs=runs)

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_vision.ipynb.
Converted 01_imdb.ipynb.
Converted 02_tabular.ipynb.
Converted 09_utils.ipynb.
Converted index.ipynb.
